# Custom TransferLearning using VGG16

In [ ]:
#!pip install scikit-learn
#!pip install imutils
#!pip install progressbar

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from pyimagesearch.io import HDF5DatasetWriter
from imutils import paths
import numpy as np
import progressbar
import argparse
import random
import os

In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Parámetros de la extracción

In [7]:
img_size = 224
batch_size = 32
buffer_size = 1000 #control the number of extracted feature store in memory
path_train = 'data_' + str(img_size) + '/train'
path_validation = 'data_' + str(img_size) + '/validation'
path_output_train_hdf5 = 'HDF5/VGG16_train.hdf5'
path_output_validation_hdf5 = 'HDF5/VGG16_validation.hdf5'

# Lectura de la data para el entrenamiento

In [8]:
#cargando las imágenes
imagePaths_train = list(paths.list_images(path_train))
imagePaths_validation = list(paths.list_images(path_validation))
#moviendo de forma aleatoria el orden de las imágenes
random.shuffle(imagePaths_train)
random.shuffle(imagePaths_validation)

#Obteniendo la etiqueta de la imagen
labels_train = [p.split(os.path.sep)[-2] for p in imagePaths_train]
#Códificando las etiquetas en númerops
le = LabelEncoder()
labels_train = le.fit_transform(labels_train)

#Obteniendo la etiqueta de la imagen
labels_validation = [p.split(os.path.sep)[-2] for p in imagePaths_validation]
#Códificando las etiquetas en númerops
le = LabelEncoder()
labels_validation = le.fit_transform(labels_validation)

# Creando el modelo

In [9]:
model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(img_size,img_size,3)
)
model.summary()

58892288/58889256 [==============================] - 7s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

# Creando el archivo hdf5

In [10]:
dataset_train = HDF5DatasetWriter((len(imagePaths_train), 512 * 7 * 7), #es igual al tamaño de la última capa
                            path_output_train_hdf5,
                            dataKey = "features",
                            bufSize = buffer_size)
dataset_train.storeClassLabels(le.classes_)

dataset_validation = HDF5DatasetWriter((len(imagePaths_validation), 512 * 7 * 7), #es igual al tamaño de la última capa
                            path_output_validation_hdf5,
                            dataKey = "features",
                            bufSize = buffer_size)
dataset_validation.storeClassLabels(le.classes_)

# Creando la extracción de la data train

In [11]:
widgets = ["Extracción de características: ",
           progressbar.Percentage(),
           " ",
           progressbar.Bar(),
           " ",
           progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=len(imagePaths_train),
                               widgets=widgets).start()

# recorriendo el array con una distancia del batch_size
for i in np.arange(0, len(imagePaths_train), batch_size):
    # obteniendo los paths y labels de las imágenes
    batchPaths = imagePaths_train[i:i + batch_size]
    batchLabels = labels_train[i:i + batch_size]
    batchImages = []
    # recorremos el bath obtenido
    for (j, imagePath) in enumerate(batchPaths):
        #cargamos las imagenes con keras
        image = load_img(imagePath, target_size=(img_size, img_size))
        image = img_to_array(image)
        
        #hacemos un pre-procesamiento de la imagen
        #le agregamos una dimensión
        image = np.expand_dims(image, axis=0)
        #obteniendo la media de la intesidad RGB del pixel
        image = imagenet_utils.preprocess_input(image)
        
        #agregando la imagen al batch
        batchImages.append(image)
        
    #creamos un stack con el array
    batchImages = np.vstack(batchImages)
    features = model.predict(batchImages, batch_size=batch_size)
    
    #hacemos un reshape de la predicción, convirtiendolo en 1D
    features = features.reshape((features.shape[0], 512 * 7 * 7))
    
    #agregamps los features en  el HDF5
    dataset_train.add(features, batchLabels)
    pbar.update(i)
        
#cerramos el  HDF5
dataset_train.close()
pbar.finish()

Extracción de características: 100% |###########################| Time: 0:01:47


In [12]:
widgets = ["Extracción de características: ",
           progressbar.Percentage(),
           " ",
           progressbar.Bar(),
           " ",
           progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=len(imagePaths_validation),
                               widgets=widgets).start()

# recorriendo el array con una distancia del batch_size
for i in np.arange(0, len(imagePaths_validation), batch_size):
    # obteniendo los paths y labels de las imágenes
    batchPaths = imagePaths_validation[i:i + batch_size]
    batchLabels = labels_validation[i:i + batch_size]
    batchImages = []
    # recorremos el bath obtenido
    for (j, imagePath) in enumerate(batchPaths):
        #cargamos las imagenes con keras
        image = load_img(imagePath, target_size=(img_size, img_size))
        image = img_to_array(image)
        
        #hacemos un pre-procesamiento de la imagen
        #le agregamos una dimensión
        image = np.expand_dims(image, axis=0)
        #obteniendo la media de la intesidad RGB del pixel
        image = imagenet_utils.preprocess_input(image)
        
        #agregando la imagen al batch
        batchImages.append(image)
        
    #creamos un stack con el array
    batchImages = np.vstack(batchImages)
    features = model.predict(batchImages, batch_size=batch_size)
    
    #hacemos un reshape de la predicción, convirtiendolo en 1D
    features = features.reshape((features.shape[0], 512 * 7 * 7))
    
    #agregamps los features en  el HDF5
    dataset_validation.add(features, batchLabels)
    pbar.update(i)
        
#cerramos el  HDF5
dataset_validation.close()
pbar.finish()

Extracción de características: 100% |###########################| Time: 0:00:06


# Entrenando un clasificador con las características

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import argparse
import pickle
import h5py

In [14]:
#leemos el HDF5
db_train = h5py.File(path_output_train_hdf5, 'r')
db_validation = h5py.File(path_output_validation_hdf5, 'r')

In [20]:
print(db_train["features"].shape)
print(db_train["labels"].shape)

(23975, 25088)
(23975,)


In [21]:
#definiendo los hiperparametros
params = {"C": [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]}
model = GridSearchCV(LogisticRegression(solver="lbfgs",
                                        multi_class="auto"),
                                        params,
                                        cv=3,
                                        n_jobs=-1)
model.fit(db_train["features"], db_train["labels"])
print("[INFO] mejores hipeparametros: {}".format(model.best_params_))

PicklingError: Could not pickle the task to send it to the workers.

## Creando un gráfico para la revisión

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.ylim([0,1.1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.ylim([0,20])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Guardando el modelo

In [ ]:
saved_model_dir = 'model_MobileNetV2'
tf.saved_model.save(model, saved_model_dir)

## Convirtiendo el modelo en tensorflow lite para el uso en mobile

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

path_tensorflow_lite = 'model_tflite/model_MobileNetV2.tflite'
with open(path_tensorflow_lite, 'wb') as f:
  f.write(tflite_model)

# Tomando 25 imágenes de validación y observando el resultado

In [ ]:
test_data = ImageClassifierDataLoader.from_folder(path_test_validation)

def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

plt.figure(figsize=(20, 20))

for i, (image, label) in enumerate(test_data.dataset.take(100)):
    ax = plt.subplot(10, 10, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image.numpy(), cmap=plt.cm.gray)

    swapped = np.moveaxis(image, 0, 1)
    arr4d = np.expand_dims(swapped, 0)
    id_predict = np.argmax(model.predict(arr4d))
    
    predict_label = "100"
    if id_predict == 1:
        predict_label = "20"
    color = get_label_color(predict_label, test_data.index_to_label[label.numpy()])
    #color = 'black'
    ax.xaxis.label.set_color(color)
    plt.xlabel('Predicted: %s' % predict_label)
plt.show()